In [10]:
import json
import pandas as pd

In [11]:
# this helps with scrolling df
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [20]:
def read_data(path, limiter):
    """
    Args:
        path: path to dataset in jsonl file format
        limiter: number of rows

    Returns:
        data: json dict, where every line is one sample as json
    """
    with open(path) as file:
        data = [json.loads(next(file)) for x in range(limiter)]
    return data


def transform_dll_imports(json_sample):
    """
    Args:
        json_sample: one sample of dataset

    Returns:
        functions_dict: dict with all dll functions name with value True
    """
    imports = sample["imports"]
    functions_dict = {}
    for key in imports.keys():
        functions = imports[key]
        functions_with_values = {key.lower() + "-" + f_name: True for f_name in functions}
        functions_dict.update(functions_with_values)
    return functions_dict


def transform_dict(json_dict):
    """
    Args:
        json_dict: json dict with nested key-value, where value is list

    Returns:
        functions_dict: dict with value as a key and default value True
    """
    result_dict = {}
    for key in json_dict.keys():
        keys = json_dict[key]
        values_with_default = {key.lower() + "-" + f_name: True for f_name in keys}
        result_dict.update(values_with_default)
    return result_dict


def transform_list(json_list):
    """
    Args:
        json_list: json list of values

    Returns:
        functions_dict: dict with all dll functions name with value True
    """
    result_dict = {}
    result_dict.update({i: True for i in json_list})
    return result_dict


def flatten_json(y, separator=''):
    """
    Args:
        y: json object
        separator: separator

    Returns:
        functions_dict: dict with flatten values
    """
    out = {}

    def flatten(x, name=separator):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [14]:
# 1. read datas
data_path = 'sample_data_6k.jsonl'
data = read_data(data_path, limiter = 6000)
# 2. iterate over dataset
flatten_dataset = []
for sample in data:
    # version 1 dummy way
    # flat = flatten_json(sample)
    # flatten_dataset.append(flat)
    # collect transformed values for each sample

    # version 2
    transformed = {}
    transformed.update({"label": sample['label']})
    transformed.update({"sha256": sample['sha256']})
    transformed.update({"md5": sample['md5']})
    transformed.update({"appeared": sample['appeared']})
    transformed.update({"avclass": sample['avclass']})
    transformed.update(flatten_json(sample['histogram'], 'histogram.'))
    transformed.update(flatten_json(sample['byteentropy'], 'byteentropy.'))
    transformed.update(transform_dict(sample["imports"]))
    transformed.update(transform_list(sample["exports"]))
    transformed.update(flatten_json(sample['general']))
    transformed.update(flatten_json(sample['strings']))
    transformed.update(flatten_json(sample['header']))
    transformed.update(flatten_json(sample['datadirectories']))

    # here we fill with transform data
    flatten_dataset.append(transformed)

# change all transformed dataset to dataframe object
df = pd.DataFrame(flatten_dataset)
df


,label,sha256,md5,appeared,avclass,histogram.0,histogram.1,histogram.2,histogram.3,histogram.4,histogram.5,histogram.6,histogram.7,histogram.8,histogram.9,histogram.10,histogram.11,histogram.12,histogram.13,histogram.14,histogram.15,histogram.16,histogram.17,histogram.18,histogram.19,histogram.20,histogram.21,histogram.22,histogram.23,histogram.24,histogram.25,histogram.26,histogram.27,histogram.28,histogram.29,histogram.30,histogram.31,histogram.32,histogram.33,histogram.34,histogram.35,histogram.36,histogram.37,histogram.38,histogram.39,histogram.40,histogram.41,histogram.42,histogram.43,histogram.44,histogram.45,histogram.46,histogram.47,histogram.48,histogram.49,histogram.50,histogram.51,histogram.52,histogram.53,histogram.54,histogram.55,histogram.56,histogram.57,histogram.58,histogram.59,histogram.60,histogram.61,histogram.62,histogram.63,histogram.64,histogram.65,histogram.66,histogram.67,histogram.68,histogram.69,histogram.70,histogram.71,histogram.72,histogram.73,histogram.74,histogram.75,histogram.76,histogram.77,histogram.78,histogram.79,histogram.80,histogram.81,histogram.82,histogram.83,histogram.84,histogram.85,histogram.86,histogram.87,histogram.88,histogram.89,histogram.90,histogram.91,histogram.92,histogram.93,histogram.94,histogram.95,histogram.96,histogram.97,histogram.98,histogram.99,histogram.100,histogram.101,histogram.102,histogram.103,histogram.104,histogram.105,histogram.106,histogram.107,histogram.108,histogram.109,histogram.110,histogram.111,histogram.112,histogram.113,histogram.114,histogram.115,histogram.116,histogram.117,histogram.118,histogram.119,histogram.120,histogram.121,histogram.122,histogram.123,histogram.124,histogram.125,histogram.126,histogram.127,histogram.128,histogram.129,histogram.130,histogram.131,histogram.132,histogram.133,histogram.134,histogram.135,histogram.136,histogram.137,histogram.138,histogram.139,histogram.140,histogram.141,histogram.142,histogram.143,histogram.144,histogram.145,histogram.146,histogram.147,histogram.148,histogram.149,histogram.150,histogram.151,histogram.152,histogram.153,histogram.154,histogram.155,histogram.156,histogram.157,histogram.158,histogram.159,histogram.160,histogram.161,histogram.162,histogram.163,histogram.164,histogram.165,histogram.166,histogram.167,histogram.168,histogram.169,histogram.170,histogram.171,histogram.172,histogram.173,histogram.174,histogram.175,histogram.176,histogram.177,histogram.178,histogram.179,histogram.180,histogram.181,histogram.182,histogram.183,histogram.184,histogram.185,histogram.186,histogram.187,histogram.188,histogram.189,histogram.190,histogram.191,histogram.192,histogram.193,histogram.194,histogram.195,histogram.196,histogram.197,histogram.198,histogram.199,histogram.200,histogram.201,histogram.202,histogram.203,histogram.204,histogram.205,histogram.206,histogram.207,histogram.208,histogram.209,histogram.210,histogram.211,histogram.212,histogram.213,histogram.214,histogram.215,histogram.216,histogram.217,histogram.218,histogram.219,histogram.220,histogram.221,histogram.222,histogram.223,histogram.224,histogram.225,histogram.226,histogram.227,histogram.228,histogram.229,histogram.230,histogram.231,histogram.232,histogram.233,histogram.234,histogram.235,histogram.236,histogram.237,histogram.238,histogram.239,histogram.240,histogram.241,histogram.242,histogram.243,histogram.244,...,PtInRegion,PtVisible,QueryFontAssocStatus,RealizePalette,RectInRegion,RectVisible,Rectangle,RemoveFontMemResourceEx,RemoveFontResourceA,RemoveFontResourceExA,RemoveFontResourceExW,RemoveFontResourceTracking,RemoveFontResourceW,ResetDCA,ResetDCW,ResizePalette,RestoreDC,RoundRect,STROBJ_bEnum,STROBJ_bEnumPositionsOnly,STROBJ_bGetAdvanceWidths,STROBJ_dwGetCodePage,STROBJ_vEnumStart,SaveDC,ScaleRgn,ScaleValues,ScaleViewportExtEx,ScaleWindowExtEx,ScriptGetFontAlternateGlyphs,ScriptGetFontFeatureTags,ScriptGetFontLanguageTags,ScriptGetFontScriptTags,ScriptItemizeOpenType,ScriptPlaceOpenType,ScriptPositionSingleGlyph,ScriptShapeOpenType,Scr

In [15]:
df.head()

,label,sha256,md5,appeared,avclass,histogram.0,histogram.1,histogram.2,histogram.3,histogram.4,histogram.5,histogram.6,histogram.7,histogram.8,histogram.9,histogram.10,histogram.11,histogram.12,histogram.13,histogram.14,histogram.15,histogram.16,histogram.17,histogram.18,histogram.19,histogram.20,histogram.21,histogram.22,histogram.23,histogram.24,histogram.25,histogram.26,histogram.27,histogram.28,histogram.29,histogram.30,histogram.31,histogram.32,histogram.33,histogram.34,histogram.35,histogram.36,histogram.37,histogram.38,histogram.39,histogram.40,histogram.41,histogram.42,histogram.43,histogram.44,histogram.45,histogram.46,histogram.47,histogram.48,histogram.49,histogram.50,histogram.51,histogram.52,histogram.53,histogram.54,histogram.55,histogram.56,histogram.57,histogram.58,histogram.59,histogram.60,histogram.61,histogram.62,histogram.63,histogram.64,histogram.65,histogram.66,histogram.67,histogram.68,histogram.69,histogram.70,histogram.71,histogram.72,histogram.73,histogram.74,histogram.75,histogram.76,histogram.77,histogram.78,histogram.79,histogram.80,histogram.81,histogram.82,histogram.83,histogram.84,histogram.85,histogram.86,histogram.87,histogram.88,histogram.89,histogram.90,histogram.91,histogram.92,histogram.93,histogram.94,histogram.95,histogram.96,histogram.97,histogram.98,histogram.99,histogram.100,histogram.101,histogram.102,histogram.103,histogram.104,histogram.105,histogram.106,histogram.107,histogram.108,histogram.109,histogram.110,histogram.111,histogram.112,histogram.113,histogram.114,histogram.115,histogram.116,histogram.117,histogram.118,histogram.119,histogram.120,histogram.121,histogram.122,histogram.123,histogram.124,histogram.125,histogram.126,histogram.127,histogram.128,histogram.129,histogram.130,histogram.131,histogram.132,histogram.133,histogram.134,histogram.135,histogram.136,histogram.137,histogram.138,histogram.139,histogram.140,histogram.141,histogram.142,histogram.143,histogram.144,histogram.145,histogram.146,histogram.147,histogram.148,histogram.149,histogram.150,histogram.151,histogram.152,histogram.153,histogram.154,histogram.155,histogram.156,histogram.157,histogram.158,histogram.159,histogram.160,histogram.161,histogram.162,histogram.163,histogram.164,histogram.165,histogram.166,histogram.167,histogram.168,histogram.169,histogram.170,histogram.171,histogram.172,histogram.173,histogram.174,histogram.175,histogram.176,histogram.177,histogram.178,histogram.179,histogram.180,histogram.181,histogram.182,histogram.183,histogram.184,histogram.185,histogram.186,histogram.187,histogram.188,histogram.189,histogram.190,histogram.191,histogram.192,histogram.193,histogram.194,histogram.195,histogram.196,histogram.197,histogram.198,histogram.199,histogram.200,histogram.201,histogram.202,histogram.203,histogram.204,histogram.205,histogram.206,histogram.207,histogram.208,histogram.209,histogram.210,histogram.211,histogram.212,histogram.213,histogram.214,histogram.215,histogram.216,histogram.217,histogram.218,histogram.219,histogram.220,histogram.221,histogram.222,histogram.223,histogram.224,histogram.225,histogram.226,histogram.227,histogram.228,histogram.229,histogram.230,histogram.231,histogram.232,histogram.233,histogram.234,histogram.235,histogram.236,histogram.237,histogram.238,histogram.239,histogram.240,histogram.241,histogram.242,histogram.243,histogram.244,...,PtInRegion,PtVisible,QueryFontAssocStatus,RealizePalette,RectInRegion,RectVisible,Rectangle,RemoveFontMemResourceEx,RemoveFontResourceA,RemoveFontResourceExA,RemoveFontResourceExW,RemoveFontResourceTracking,RemoveFontResourceW,ResetDCA,ResetDCW,ResizePalette,RestoreDC,RoundRect,STROBJ_bEnum,STROBJ_bEnumPositionsOnly,STROBJ_bGetAdvanceWidths,STROBJ_dwGetCodePage,STROBJ_vEnumStart,SaveDC,ScaleRgn,ScaleValues,ScaleViewportExtEx,ScaleWindowExtEx,ScriptGetFontAlternateGlyphs,ScriptGetFontFeatureTags,ScriptGetFontLanguageTags,ScriptGetFontScriptTags,ScriptItemizeOpenType,ScriptPlaceOpenType,ScriptPositionSingleGlyph,ScriptShapeOpenType,Scr

In [1]:
df.shape

NameError: name 'df' is not defined

In [19]:
df.columns

Index(['label', 'sha256', 'md5', 'appeared', 'avclass', 'histogram.0', 'histogram.1', 'histogram.2', 'histogram.3', 'histogram.4',
       ...
       '_cgo_panic', '_cgo_topofstack', 'callbackTrampoline', 'commitHookTrampoline', 'compareTrampoline', 'crosscall2', 'doneTrampoline', 'rollbackHookTrampoline', 'stepTrampoline', 'updateHookTrampoline'], dtype='object', length=239191)

In [16]:
header = df.loc[:,df.columns.str.startswith("header")]

In [17]:
header.head()

,header_word
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
